**GOALS**:

- Discover spark and use it locally on a small subset of images from the [fruits_360 dataset](https://www.kaggle.com/datasets/moltean/fruits).
- Extract image features and reduce its dimensionality with PCA.

**Table of contents**<a id='toc0_'></a>    
- 1. [Define paths and build a small subset of images to experiment with my local machine and avoid huge cost on the cloud.](#toc1_)    
- 2. [Launch a Spark Session](#toc2_)    
- 3. [Load Images](#toc3_)    
- 4. [Build model for feature extraction and broadcast its weights.](#toc4_)    
- 5. [Functions](#toc5_)    
- 6. [Loading results computed locally with spark](#toc6_)    
- 7. [Compare to not distributed computation based on tf and sklearn.](#toc7_)    
- 8. [?](#toc8_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
import pandas as pd
from PIL import Image
import numpy as np
import io
import os
import shutil
import random

import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, element_at, split
from pyspark.sql.functions import udf, pandas_udf, PandasUDFType
from pyspark.sql.types import *
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import StandardScaler, PCA
from pyspark.ml import Pipeline

from typing import Iterator

2023-07-18 09:51:14.711840: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-18 09:51:15.236535: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-18 09:51:15.238514: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-18 09:51:16.732699: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# 1.  [&#9650;](#toc0_) <a id='toc1_'></a>Define paths and build a small subset of images to experiment with my local machine and avoid huge cost on the cloud.

In [2]:
PATH = os.getcwd()
DATA_PATH = os.path.join(PATH ,'fruits_360')
RESULT_PATH = os.path.join(PATH ,'Results')
TRAIN_PATH = os.path.join(DATA_PATH , 'Training')
TEST_PATH = os.path.join(DATA_PATH , 'Test')
IMAGE_SUBSET_PATH = os.path.join(PATH, 'images_subset')

if not os.path.exists(RESULT_PATH):
    os.mkdir(RESULT_PATH)

if not os.path.exists(IMAGE_SUBSET_PATH):
    os.mkdir(IMAGE_SUBSET_PATH)

print('PATH: '+ PATH \
      +'\nDATA_PATH: '+  DATA_PATH \
      +'\nRESULT_PATH: '+RESULT_PATH
        +'\nTRAIN_PATH: '+  TRAIN_PATH \
      +'\nTEST_PATH: '+TEST_PATH)

PATH: /home/louberehc/OCR/projets/8_cloud_computing
DATA_PATH: /home/louberehc/OCR/projets/8_cloud_computing/fruits_360
RESULT_PATH: /home/louberehc/OCR/projets/8_cloud_computing/Results
TRAIN_PATH: /home/louberehc/OCR/projets/8_cloud_computing/fruits_360/Training
TEST_PATH: /home/louberehc/OCR/projets/8_cloud_computing/fruits_360/Test


In [3]:
!tree /home/louberehc/OCR/projets/8_cloud_computing/fruits_360/Training -L 1

/home/louberehc/OCR/projets/8_cloud_computing/fruits_360/Training
├── Apple Braeburn
├── Apple Crimson Snow
├── Apple Golden 1
├── Apple Golden 2
├── Apple Golden 3
├── Apple Granny Smith
├── Apple Pink Lady
├── Apple Red 1
├── Apple Red 2
├── Apple Red 3
├── Apple Red Delicious
├── Apple Red Yellow 1
├── Apple Red Yellow 2
├── Apricot
├── Avocado
├── Avocado ripe
├── Banana
├── Banana Lady Finger
├── Banana Red
├── Beetroot
├── Blueberry
├── Cactus fruit
├── Cantaloupe 1
├── Cantaloupe 2
├── Carambula
├── Cauliflower
├── Cherry 1
├── Cherry 2
├── Cherry Rainier
├── Cherry Wax Black
├── Cherry Wax Red
├── Cherry Wax Yellow
├── Chestnut
├── Clementine
├── Cocos
├── Corn
├── Corn Husk
├── Cucumber Ripe
├── Cucumber Ripe 2
├── Dates
├── Eggplant
├── Fig
├── Ginger Root
├── Granadilla
├── Grape Blue
├── Grapefruit Pink
├── Grapefruit White
├── Grape Pink
├── Grape White
├── Grape White 2
├── Grape White 3
├── Grape White 4
├── Guava
├── Hazelnut
├── Huckleberry
├── Kaki
├── Kiwi
├── Kohlra

In [4]:
!tree /home/louberehc/OCR/projets/8_cloud_computing/fruits_360/Test -L 1


/home/louberehc/OCR/projets/8_cloud_computing/fruits_360/Test
├── Apple Braeburn
├── Apple Crimson Snow
├── Apple Golden 1
├── Apple Golden 2
├── Apple Golden 3
├── Apple Granny Smith
├── Apple Pink Lady
├── Apple Red 1
├── Apple Red 2
├── Apple Red 3
├── Apple Red Delicious
├── Apple Red Yellow 1
├── Apple Red Yellow 2
├── Apricot
├── Avocado
├── Avocado ripe
├── Banana
├── Banana Lady Finger
├── Banana Red
├── Beetroot
├── Blueberry
├── Cactus fruit
├── Cantaloupe 1
├── Cantaloupe 2
├── Carambula
├── Cauliflower
├── Cherry 1
├── Cherry 2
├── Cherry Rainier
├── Cherry Wax Black
├── Cherry Wax Red
├── Cherry Wax Yellow
├── Chestnut
├── Clementine
├── Cocos
├── Corn
├── Corn Husk
├── Cucumber Ripe
├── Cucumber Ripe 2
├── Dates
├── Eggplant
├── Fig
├── Ginger Root
├── Granadilla
├── Grape Blue
├── Grapefruit Pink
├── Grapefruit White
├── Grape Pink
├── Grape White
├── Grape White 2
├── Grape White 3
├── Grape White 4
├── Guava
├── Hazelnut
├── Huckleberry
├── Kaki
├── Kiwi
├── Kohlrabi
├

In [5]:
# Copy some random images from the training dataset in the subset dir.
# 10 images per fruits among the 5 first fruits appearing in listdir.
n_images = 10
n_fruit_types = 32

for fruit_name in os.listdir(TRAIN_PATH)[:n_fruit_types]:
    train_fruit_path = os.path.join(TRAIN_PATH, fruit_name)
    subset_fruit_path = os.path.join(IMAGE_SUBSET_PATH, fruit_name)
    
    if not os.path.exists(subset_fruit_path):
        os.mkdir(subset_fruit_path)
    
    random_relative_filenames = random.sample(
        os.listdir(train_fruit_path),
        k=n_images
    )

    for fn in random_relative_filenames:
        shutil.copy(
            os.path.join(train_fruit_path, fn),
            os.path.join(subset_fruit_path, fn)
        )

In [6]:
!tree images_subset

images_subset
├── Apple Golden 2
│   ├── 20_100.jpg
│   ├── 284_100.jpg
│   ├── 313_100.jpg
│   ├── 320_100.jpg
│   ├── r_170_100.jpg
│   ├── r_187_100.jpg
│   ├── r_238_100.jpg
│   ├── r_289_100.jpg
│   ├── r_305_100.jpg
│   └── r_318_100.jpg
├── Apple Pink Lady
│   ├── 121_100.jpg
│   ├── 58_100.jpg
│   ├── 9_100.jpg
│   ├── r_162_100.jpg
│   ├── r_167_100.jpg
│   ├── r_188_100.jpg
│   ├── r_195_100.jpg
│   ├── r_298_100.jpg
│   ├── r_301_100.jpg
│   └── r_43_100.jpg
├── Apple Red 1
│   ├── 156_100.jpg
│   ├── 205_100.jpg
│   ├── 254_100.jpg
│   ├── 258_100.jpg
│   ├── 319_100.jpg
│   ├── r_101_100.jpg
│   ├── r_132_100.jpg
│   ├── r_18_100.jpg
│   ├── r_181_100.jpg
│   └── r_223_100.jpg
├── Cherry 1
│   ├── 117_100.jpg
│   ├── 162_100.jpg
│   ├── 271_100.jpg
│   ├── 288_100.jpg
│   ├── 303_100.jpg
│   ├── 31_100.jpg
│   ├── r_141_100.jpg
│   ├── r_19_100.jpg
│   ├── r_25_100.jpg
│   └── r_298_100.jpg
├── Cherry Wax Red
│   ├── 187_100.jpg
│   ├── 304_100.jpg
│   ├── 308_100.jpg
│   

# 2.  [&#9650;](#toc0_) <a id='toc2_'></a>Launch a Spark Session

Note:

First, I had errors of modules not found when using PandasUDF.

To avoid that, I had to create a conda environment (p8_env) with packages called in such functions (tensorflow, numpy...) , conda-pack it, and provide
it to the spark session. Click [here](https://spark.apache.org/docs/latest/api/python/user_guide/python_packaging.html) to see the ad-hoc documentation.

This makes the session much longer to open because the dependencies are installed on the driver and the executors, but then, it works.

It introduces some new problems with the heartbeater, but I neglect it for now.

In [3]:
os.environ['PYSPARK_PYTHON'] = "/home/louberehc/miniconda3/envs/p8_env/bin/python" 
os.environ['PYSPARK_DRIVER_PYTHON'] = "/home/louberehc/miniconda3/envs/p8_env/bin/python" 

spark = (
    SparkSession
    .builder
    .appName('P8')
    .master('local[*]')
    .config("spark.sql.parquet.writeLegacyFormat", 'true')
    .config(
        "spark.archives",  # 'spark.yarn.dist.archives' in YARN.
        "p8_env.tar.gz#environment"
    )
    .getOrCreate()
)

23/07/17 20:01:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/17 20:02:16 ERROR Inbox: Ignoring error
java.lang.NullPointerException
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$register(BlockManagerMasterEndpoint.scala:600)
	at org.apache.spark.storage.BlockManagerMasterEndpoint$$anonfun$receiveAndReply$1.applyOrElse(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:103)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.process(Inbox.scala:100)
	at org.apache.spark.rpc.netty.MessageLoop.org$apache$spark$rpc$netty$MessageLoop$$receiveLoop(MessageLoop.scala:75)
	at org.apache.spark.rpc.netty.MessageLoop$$

In [4]:
sc = spark.sparkContext

In [5]:
spark

# 3.  [&#9650;](#toc0_) <a id='toc3_'></a>Load Images

In [6]:
images = spark.read.format("binaryFile") \
    .option("pathGlobFilter", "*.jpg") \
    .option("recursiveFileLookup", "true") \
    .load(IMAGE_SUBSET_PATH)
    
# Add a label column from the image path
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))

In [7]:
images.printSchema()

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)



In [12]:
images.count()

500

In [13]:
# Inspect path and label
print(images.select('path','label').show(10,False))

+--------------------------------------------------------------------------------------+---------+
|path                                                                                  |label    |
+--------------------------------------------------------------------------------------+---------+
|file:/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Raspberry/136_100.jpg|Raspberry|
|file:/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Raspberry/11_100.jpg |Raspberry|
|file:/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Raspberry/39_100.jpg |Raspberry|
|file:/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Raspberry/325_100.jpg|Raspberry|
|file:/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Raspberry/116_100.jpg|Raspberry|
|file:/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Pineapple/112_100.jpg|Pineapple|
|file:/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Pineapple/160_100.jpg|Pineapple|
|file:/hom

In [14]:
print(images.select('content').show(1,False))

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# 4.  [&#9650;](#toc0_) <a id='toc4_'></a>Build model for feature extraction and broadcast its weights.

In [8]:
# DL or load mobilenetV2
model = MobileNetV2(
    weights='imagenet',
    include_top=True,
    input_shape=(224, 224, 3)
)

2023-07-17 20:02:32.419677: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [9]:
# Create a model without the last layer
new_model = Model(
    inputs=model.input,
    outputs=model.layers[-2].output
)

In [10]:
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [10]:
# Make a broadcast object to broadcast the model weights to
# each worker.
broadcast_weights = sc.broadcast(new_model.get_weights())

In [11]:
broadcast_weights

In [12]:
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

# 5.  [&#9650;](#toc0_) <a id='toc5_'></a>Functions

In [13]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and set weights to the broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    
    for layer in model.layers:
        layer.trainable = False
        
    new_model = Model(inputs=model.input,
                      outputs=model.layers[-2].output)
    new_model.set_weights(broadcast_weights.value)
    return new_model


def preprocess(content):
    """
    - Takes raw image bytes and transforms it to a pillow image.
    - Resize the images to the Mobilenet input size 
    and preprocess the pixel values.    
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, the output features can be multi-dimensional tensors.
    # Thus, we flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)


@pandas_udf('array<float>')
def featurize_udf(
    content_series_iter: Iterator[pd.Series]
) -> Iterator[pd.Series]:
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).
    
    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                                is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

In [14]:
# Build a DF with the path, label and features of each image
features_df = (
    images
    .repartition(300)
    .select(
        col("path"),
        col("label"),
        featurize_udf("content").alias("features")
   )
)

#MLLib needs some post processing of the features column format
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

features_df = (
    features_df
    .select(
        col("path"),  
        col("label"),
        list_to_vector_udf(features_df["features"]).alias("features")
   )
)

# Define a pipeline to Standardize the features
# and compute the PCA projection onto the 300 first PCs.
scaler = StandardScaler(
    inputCol="features",
    outputCol="scaled_features",
    withStd=True,
    withMean=True
)

pca = PCA(
    k=300,
    inputCol=scaler.getOutputCol(),
    outputCol="pca_features",
)

pipeline = Pipeline(stages=[scaler , pca])
model = pipeline.fit(features_df)
features_df = model.transform(features_df)

# Write results
(
    features_df
    .drop('scaled_features')
    .write.mode("overwrite")
    .parquet(RESULT_PATH)
)

2023-07-17 20:02:46.092787: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-17 20:02:46.367748: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-17 20:02:46.463441: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-17 20:02:46.502024: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-17 20:02:46.564929: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-17 20:02:46.651930: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-17 20:02:46.666632: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-17 20:02:46.866554: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-

I am not going for optimization right now, but it is not fast... It took ~17 minutes to compute the features and its PCA projection for 50 images.

# 6.  [&#9650;](#toc0_) <a id='toc6_'></a>Loading results computed locally with spark

In [33]:
df = pd.read_parquet(RESULT_PATH, engine='pyarrow')
df.shape

(320, 4)

In [34]:
df.columns

Index(['path', 'label', 'features', 'pca_features'], dtype='object')

In [35]:
df.loc[0, 'features']["values"].shape

(1280,)

In [36]:
df.loc[0, 'pca_features']["values"].shape

(300,)

In [37]:
df.loc[0, 'pca_features']["values"]

array([ 6.25521841e+00,  1.60126250e+00,  4.83094892e+00, -4.08174443e+00,
       -2.16043178e+00,  9.56921187e+00,  2.19970715e-01, -6.20291185e+00,
        3.08297696e+00,  5.65644473e+00, -4.72838495e+00,  1.09280383e+00,
       -3.52825095e+00,  1.80234683e-01, -7.04883818e-01,  4.82799149e+00,
       -2.03436723e+00,  1.76621269e+00,  1.13826690e+00,  9.29235301e+00,
        4.20503896e-01,  1.89228250e-01, -6.25211537e-01, -2.04012120e+00,
        1.95053092e+00, -1.35374726e+00,  3.48203758e+00,  1.95097982e+00,
       -8.17487136e-01,  9.59395738e-01,  2.18300538e+00,  1.03877231e+00,
        5.70958835e-01,  3.34160743e+00, -2.08947962e+00,  1.44429154e+00,
        8.11047316e-01,  3.37691675e-01,  1.99258905e+00, -1.42532019e+00,
        1.53364691e+00, -3.89652096e+00,  4.02096186e+00, -2.79418830e+00,
       -4.75675641e-01,  1.65751112e+00, -1.99288612e+00, -4.74866438e-01,
        7.01606043e-01,  6.76222252e-01,  3.61652621e-01,  1.50028329e+00,
        2.07130259e+00, -

Results have the right dimension.

In [38]:
df.loc[0, 'path']

'file:/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Granadilla/269_100.jpg'

# 7.  [&#9650;](#toc0_) <a id='toc7_'></a>Compare to not distributed computation based on tf and sklearn.

In [73]:
PATH_NON_DIST = './non-distributed-results/'
fts_non_dist = pd.read_csv(PATH_NON_DIST + 'fts.csv.gz', index_col=0)
scaled_non_dist = pd.read_csv(PATH_NON_DIST + 'scaled.csv.gz', index_col=0)
pca_non_dist = pd.read_csv(PATH_NON_DIST + 'pca.csv.gz', index_col=0)

In [74]:
fts_non_dist.head()

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature1270,feature1271,feature1272,feature1273,feature1274,feature1275,feature1276,feature1277,feature1278,feature1279
/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Kumquats/71_100.jpg,0.036694,0.0,0.0,0.0,0.005719,0.028472,0.058900,0.028014,0.000000,0.000000,...,0.000000,0.0,0.000000,0.003670,0.038330,0.000000,0.0,0.0,0.034528,0.0
/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Kumquats/r_214_100.jpg,0.309548,0.0,0.0,0.0,0.000009,0.000000,1.155231,0.000000,0.145102,0.039393,...,0.064710,0.0,0.000000,0.030180,0.043894,0.088751,0.0,0.0,0.004618,0.0
/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Kumquats/r_234_100.jpg,0.092173,0.0,0.0,0.0,0.000000,0.000000,0.620752,0.016290,0.022272,0.025216,...,0.000000,0.0,0.000000,0.000000,0.184897,0.013192,0.0,0.0,0.000000,0.0
/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Kumquats/r_219_100.jpg,0.327651,0.0,0.0,0.0,0.014103,0.000000,0.602290,0.000000,0.078213,0.032840,...,0.138702,0.0,0.000000,0.191602,0.023143,0.006719,0.0,0.0,0.000000,0.0
/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Kumquats/21_100.jpg,0.294717,0.0,0.0,0.0,0.023944,0.012229,0.219464,0.019068,0.000000,0.000000,...,0.023362,0.0,0.005253,0.047513,0.179043,0.000000,0.0,0.0,0.134759,0.0


In [75]:
df.head()

,path,label,features,pca_features
0,file:/home/louberehc/OCR/projets/8_cloud_compu...,Granadilla,"{'type': 1, 'size': None, 'indices': None, 'va...","{'type': 1, 'size': None, 'indices': None, 'va..."
1,file:/home/louberehc/OCR/projets/8_cloud_compu...,Granadilla,"{'type': 1, 'size': None, 'indices': None, 'va...","{'type': 1, 'size': None, 'indices': None, 'va..."
2,file:/home/louberehc/OCR/projets/8_cloud_compu...,Maracuja,"{'type': 1, 'size': None, 'indices': None, 'va...","{'type': 1, 'size': None, 'indices': None, 'va..."
3,file:/home/louberehc/OCR/projets/8_cloud_compu...,Grape White 3,"{'type': 1, 'size': None, 'indices': None, 'va...","{'type': 1, 'size': None, 'indices': None, 'va..."
4,file:/home/louberehc/OCR/projets/8_cloud_compu...,Maracuja,"{'type': 1, 'size': None, 'indices': None, 'va...","{'type': 1, 'size': None, 'indices': None, 'va..."


In [76]:
name_ = df.loc[0, 'path']
c_name_ = name_.removeprefix('file:')
c_name_

'/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Granadilla/269_100.jpg'

In [84]:
fts_ = df.loc[0, 'features']['values']
fts_[:25]

array([1.04994011, 0.01587156, 0.        , 0.        , 0.        ,
       0.92849171, 0.59414637, 0.00505837, 0.2336525 , 0.00953782,
       0.02596591, 0.06156127, 0.        , 0.20102984, 0.        ,
       0.07110962, 0.        , 0.32307297, 0.01995695, 0.21698575,
       0.        , 0.06414215, 0.        , 0.03265156, 0.        ])

In [85]:
fts_non_dist.loc[c_name_].values[:25]

array([0.8266986 , 0.0335313 , 0.        , 0.        , 0.        ,
       1.2652407 , 0.59311146, 0.        , 0.13892299, 0.00309227,
       0.06322599, 0.06061769, 0.        , 0.26635894, 0.        ,
       0.06027289, 0.        , 0.29231146, 0.00842814, 0.3289087 ,
       0.        , 0.10575527, 0.        , 0.04254452, 0.        ])

There are differences, but the magnitude is quite similar and 0 are in the same position.

This is mainly due to little differences in the preprocessing stage, supposedly with resampling (see [this notebook](./test-no-distribution.ipynb))

# 8.  [&#9650;](#toc0_) <a id='toc8_'></a>?

In [44]:
ONLINE_RESULT_PATH = os.getcwd() + '/online_results'
dfo = pd.read_parquet(ONLINE_RESULT_PATH, engine='pyarrow')
dfo.shape

(50, 4)

In [68]:
search = True
n_elem = 0

while search:
    print(n_elem)
    name_ = dfo.loc[n_elem,'path']
    search_str = '/'.join(name_.split('/')[-2:])
    if fts_non_dist.index.str.contains(search_str).any():
        search = False
    else:
        n_elem += 1
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


KeyError: 50

In [57]:
c_name_ = (
    '/home/louberehc/OCR/projets/8_cloud_computing'
    + name_.removeprefix('s3://oc-cloud-computing')
)
c_name_

'/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Kumquats/r_128_100.jpg'

In [70]:
fts_non_dist.loc[fts_non_dist.index.str.contains('Kumquats')]

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature1270,feature1271,feature1272,feature1273,feature1274,feature1275,feature1276,feature1277,feature1278,feature1279
/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Kumquats/71_100.jpg,0.036694,0.000000,0.0,0.0,0.005719,0.028472,0.058900,0.028014,0.000000,0.000000,...,0.000000,0.0,0.000000,0.003670,0.038330,0.000000,0.0,0.0,0.034528,0.0
/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Kumquats/r_214_100.jpg,0.309548,0.000000,0.0,0.0,0.000009,0.000000,1.155231,0.000000,0.145102,0.039393,...,0.064710,0.0,0.000000,0.030180,0.043894,0.088751,0.0,0.0,0.004618,0.0
/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Kumquats/r_234_100.jpg,0.092173,0.000000,0.0,0.0,0.000000,0.000000,0.620752,0.016290,0.022272,0.025216,...,0.000000,0.0,0.000000,0.000000,0.184897,0.013192,0.0,0.0,0.000000,0.0
/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Kumquats/r_219_100.jpg,0.327651,0.000000,0.0,0.0,0.014103,0.000000,0.602290,0.000000,0.078213,0.032840,...,0.138702,0.0,0.000000,0.191602,0.023143,0.006719,0.0,0.0,0.000000,0.0
/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Kumquats/21_100.jpg,0.294717,0.000000,0.0,0.0,0.023944,0.012229,0.219464,0.019068,0.000000,0.000000,...,0.023362,0.0,0.005253,0.047513,0.179043,0.000000,0.0,0.0,0.134759,0.0
/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Kumquats/r_123_100.jpg,0.252187,0.000000,0.0,0.0,0.068111,0.075082,0.531213,0.000000,0.065831,0.015173,...,0.000000,0.0,0.007191,0.528919,0.379969,0.000000,0.0,0.0,0.238630,0.0
/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Kumquats/r_316_100.jpg,0.857297,0.000997,0.0,0.0,0.001122,0.000000,1.256253,0.213074,0.004844,0.000000,...,0.000000,0.0,0.000000,0.026232,0.034188,0.047997,0.0,0.0,0.000000,0.0
/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Kumquats/r_19_100.jpg,0.555136,0.000000,0.0,0.0,0.000000,0.005636,1.329911,0.204190,0.014705,0.000000,...,0.000000,0.0,0.000000,0.015190,0.023478,0.000000,0.0,0.0,0.000000,0.0
/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Kumquats/162_100.jpg,0.253767,0.000000,0.0,0.0,0.026469,0.012183,0.612994,0.000000,0.000000,0.005717,...,0.017938,0.0,0.000000,0.162961,0.169570,0.000000,0.0,0.0,0.045466,0.0
/home/louberehc/OCR/projets/8_cloud_computing/images_subset/Kumquats/r_65_100.jpg,0.647553,0.013179,0.0,0.0,0.007197,0.000000,0.535789,0.009908,0.147036,0.021713,...,0.000752,0.0,0.000000,0.175637,0.002533,0.101531,0.0,0.0,0.033047,0.0
